In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import json
import codecs

In [2]:
def getID():
    game_ids = json.load(codecs.open("ID.txt", 'r', 'utf-8'))
    return game_ids

In [37]:
def getReview(game_id):
    content = {}
    reviewCount = 0
    pageCount = 1
    page = "https://steamcommunity.com/app/"+game_id+"/negativereviews/?browsefilter=trendsixmonths&snr=1_5_100010_&p=1&filterLanguage=tchinese"
    options = webdriver.FirefoxOptions()
    options.add_argument('lang=zh_TW.UTF-8')              #設定成繁體中文
    driver = webdriver.Firefox(options=options)
    driver.get(page)
    if("error_box" in driver.page_source): #若需要登入
            loginPage = driver.find_element_by_class_name("global_action_link")#點擊登入
            loginPage.click()
            accountBar = driver.find_element_by_id("input_username")           #輸入帳號密碼    
            accountBar.send_keys(account)
            passwordBar = driver.find_element_by_id("input_password")
            passwordBar.send_keys(password)
            passwordBar.send_keys(Keys.ENTER)
            time.sleep(5)
            goIn = driver.find_element_by_css_selector("[class='btnv6_blue_hoverfade btn_medium']")#點擊檢視頁面(class名稱有空白)
            goIn.click()
        
    if("agegate_wizard"in driver.page_source): 
        if("ageYear" in driver.page_source):
            selectorYear = Select(driver.find_element_by_id("ageYear"))
            selectorYear.select_by_value("1995")
        goIn = driver.find_element_by_css_selector("[class='btnv6_blue_hoverfade btn_medium']")#點擊檢視頁面(class名稱有空白)
        goIn.click()
        time.sleep(5)
    time.sleep(5)
    soup = bs(driver.page_source,"html.parser")
    name = soup.find('div',{'class':"apphub_AppName ellipsis"}).text.strip()
    while(reviewCount<500):
        nowpage = soup.find('div',{'id':'page'+str(pageCount)})
        for review in nowpage.find_all('div',{'class':'apphub_CardTextContent'}):
            content[reviewCount] = review.text.strip()
            reviewCount+=1
        driver.execute_script("window.scrollBy(0,2500)") #滾動
        time.sleep(5)
        soup = bs(driver.page_source,"html.parser")
        pageCount+=1
    driver.close()
    jsons = json.dumps(content,ensure_ascii=False) 
    f = codecs.open(name+".txt", 'w', 'utf_8_sig')
    f.write(jsons)
    f.close

In [38]:
if __name__ == '__main__':
    game_ids = getID()
    count = 0
    for game_id in game_ids:
        getReview(str(game_id))
        count+=1
        if(count>=1):
            break;